In [1]:
import gzip
import math

In [2]:
def dist(x1,y1,x2,y2):
    """
    distance: euclidean distance between (x1,y1) and (x2,y2)
    """
    
    return math.sqrt((x2-x1)**2 + (y2-y1)**2)

def distL2(x1,y1,x2,y2):
    """
    Compute the L2-norm (Euclidean) distance between two points.

    The distance is rounded to the closest integer, for compatibility
    with the TSPLIB convention.

    The two points are located on coordinates (x1,y1) and (x2,y2), sent as parameters
    """
    
    xdiff = x2 - x1
    ydiff = y2 - y1

    return int(math.sqrt(xdiff*xdiff + ydiff*ydiff) + .5)


def distL1(x1,y1,x2,y2):
    """
    Compute the L1-norm (Manhattan) distance between two points.

    The distance is rounded to the closest integer, for compatibility
    with the TSPLIB convention.

    The two points are located on coordinates (x1,y1) and (x2,y2), sent as parameters
    """
    
    return int(abs(x2-x1) + abs(y2-y1)+.5)


def distLinf(x1,y1,x2,y2):
    """
    Compute the Linfty distance between two points (see TSPLIB documentation)
    """
    
    return int(max(abs(x2-x1),abs(y2-y1)))

def distATT(x1,y1,x2,y2):
    """
    Compute the ATT distance between two points (see TSPLIB documentation)
    """
    
    xd = x2 - x1
    yd = y2 - y1
    rij = math.sqrt((xd*xd + yd*yd) /10.)
    tij = int(rij + .5)
    if tij < rij:
        return tij + 1
    else:
        return tij

def distCEIL2D(x1,y1,x2,y2):
    xdiff = x2 - x1
    ydiff = y2 - y1
    
    return int(math.ceil(math.sqrt(xdiff*xdiff + ydiff*ydiff)))

def read_explicit_lowerdiag(f,n):
    c = {}
    i,j = 1,1
    while True:
        line = f.readline()
        for data in line.split():
            c[j,i] = int(data)
            j += 1
            if j>i:
                i += 1
                j = 1
            if i > n:
                return range(1,n+1),c,None,None

def read_explicit_upper(f,n):
    c = {}
    i,j = 1,2
    while True:
        line = f.readline()
        for data in line.split():
            c[i,j] = int(data)
            j += 1
            if j>n:
                i += 1
                j = i+1
            if i == n:
                return range(1,n+1),c,None,None

def read_explicit_upperdiag(f,n):
    c = {}
    i,j = 1,1
    while True:
        line = f.readline()
        for data in line.split():
            c[i,j] = int(data)
            j += 1
            if j>n:
                i += 1
                j = i
            if i == n:
                return range(1,n+1),c,None,None

def read_explicit_matrix(f,n):
    c = {}
    i,j = 1,1
    while True:
        line = f.readline()
        for data in line.split():
            if j>i:
                c[i,j] = int(data)
            j += 1
            if j>n:
                i += 1
                j = 1
            if i == n:
                return range(1,n+1),c,None,None
        

In [3]:
def read_tsplib(filename):
    "basic function for reading a symmetric problem in the TSPLIB format"
    "data is stored in an upper triangular matrix"
    "NOTE: some distance types are not handled yet"

    if filename[-3:] == ".gz":
        f = gzip.open(filename)
    else:
        f = open(filename)

    line = f.readline()
    while line.find("DIMENSION") == -1:
        line = f.readline()
    n = int(line.split()[-1])

    while line.find("EDGE_WEIGHT_TYPE") == -1:
        line = f.readline()

    if line.find("EUC_2D") != -1:
        dist = distL2
    elif line.find("MAN_2D") != -1:
        dist = distL1
    elif line.find("MAX_2D") != -1:
        dist = distLinf
    elif line.find("ATT") != -1:
        dist = distATT
    elif line.find("CEIL_2D") != -1:
        dist = distCEIL2D
    elif line.find("EXPLICIT") != -1:
        while line.find("EDGE_WEIGHT_FORMAT") == -1:
            line = f.readline()
        if line.find("LOWER_DIAG_ROW") != -1:
            while line.find("EDGE_WEIGHT_SECTION") == -1:
                line = f.readline()
            return read_explicit_lowerdiag(f,n)
        if line.find("UPPER_ROW") != -1:
            while line.find("EDGE_WEIGHT_SECTION") == -1:
                line = f.readline()
            return read_explicit_upper(f,n)
        if line.find("UPPER_DIAG_ROW") != -1:
            while line.find("EDGE_WEIGHT_SECTION") == -1:
                line = f.readline()
            return read_explicit_upperdiag(f,n)
        if line.find("FULL_MATRIX") != -1:
            while line.find("EDGE_WEIGHT_SECTION") == -1:
                line = f.readline()
            return read_explicit_matrix(f,n)
        print("error reading line " + line)
        raise(Exception)
    else:
        print("cannot deal with '%s' distances" % line)
        raise Exception

    while line.find("NODE_COORD_SECTION") == -1:
        line = f.readline()

    x,y = {},{}
    while 1:
        line = f.readline()
        if line.find("EOF") != -1 or not line: break
        (i,xi,yi) = line.split()
        x[i] = float(xi)
        y[i] = float(yi)

    V = x.keys()

    c = {}      # dictionary to hold n times n matrix
    for i in V:
        for j in V:
            c[i,j] = dist(x[i],y[i],x[j],y[j])

    return V, c, x, y

In [4]:
def main(file_name):

    V,c,x,y = read_tsplib(file_name)

    print(len(V), "vertices,", len(c), "arcs")
    
    #print "distance matrix:"
    for i in V:
        for j in V:
            if j > i:
                print(i,j,c[i,j])
            elif j < i:
                print(j,i,c[j,i])
            else:
                print("0"),
        print
    print

In [5]:
if __name__== "__main__" :
	
    file_name = "instances/tsplib/a280.tsp"
    
    main(file_name)

280 vertices, 78400 arcs
0
1 2 20
1 3 24
1 4 33
1 5 33
1 6 43
1 7 56
1 8 63
1 9 61
1 10 71
1 11 79
1 12 86
1 13 94
1 14 102
1 15 93
1 16 100
1 17 116
1 18 124
1 19 132
1 20 140
1 21 148
1 22 141
1 23 126
1 24 118
1 25 134
1 26 149
1 27 157
1 28 165
1 29 172
1 30 184
1 31 184
1 32 185
1 33 199
1 34 208
1 35 224
1 36 225
1 37 233
1 38 232
1 39 232
1 40 232
1 41 232
1 42 233
1 43 234
1 44 250
1 45 249
1 46 248
1 47 248
1 48 248
1 49 248
1 50 249
1 51 257
1 52 256
1 53 256
1 54 256
1 55 256
1 56 257
1 57 258
1 58 259
1 59 251
1 60 235
1 61 236
1 62 245
1 63 253
1 64 261
1 65 263
1 66 271
1 67 277
1 68 275
1 69 283
1 70 285
1 71 286
1 72 288
1 73 290
1 74 292
1 75 295
1 76 287
1 77 297
1 78 300
1 79 284
1 80 278
1 81 275
1 82 272
1 83 269
1 84 267
1 85 265
1 86 257
1 87 259
1 88 262
1 89 267
1 90 266
1 91 269
1 92 273
1 93 284
1 94 292
1 95 299
1 96 302
1 97 295
1 98 288
1 99 280
1 100 271
1 101 267
1 102 263
1 103 259
1 104 256
1 105 249
1 106 241
1 107 238
1 108 253
1 109 259
1 110 250
1 

In [ ]:
if __name__ == "__main__":
    import sys
    # Parse argument
    if len(sys.argv) < 2:
        print('Usage: %s instance' % sys.argv[0])
        exit(1)

    V,c,x,y = read_tsplib(sys.argv[1])
    print(len(V), "vertices,", len(c), "arcs")
    #print "distance matrix:"
    for i in V:
        for j in V:
            if j > i:
                print(c[i,j])
            elif j < i:
                print(c[j,i])
            else:
                print("0"),
        print
    print